# Modelado de lenguaje --> Language Modeling

## ¿Qué es?

En esta parte veremos una de las tareas más complejas de NLP. La generación de texto. Es decir, si queremos tener un asistente virtual, pero no queremos programarle las respuestas, ¿Qué opciones tenemos? Una de ellas es Language Modeling. De todas formas esto último aún no ha funcionado bien nunca, así que vamos a lo que sí ha funcionado? El algoritmo que predice o os sugiere la siguiente palabra en vuestro teclado? Language Modeling. ¿Os acordáis cuándo aplicamos la distancia de edición en palabras? ¿Que pasa si queremos aplicar distancias de edición pero las sugerencias de la edición estan a la misma distáncia? Podríamos aplicar Language Modeling y ver cuál es la mejor opción. 

Técnicamente, el modelado de lenguaje es una distribución de probabilidades sobre secuencias de palabras. Que significa esto?

![](https://i.imgur.com/OWHNbj9.jpg)

![](https://i.imgur.com/V9jXP6J.jpg)

## Como lo hacemos?
Es decir, es una manera de asignar una probabilidad a auna frase o texto. Como lo hacemos? Con Bayes. Tenemos los conteos hechos de bayes, solo hay que aplicar la asumpción de independéncia.

### Dependencia 
![](https://i.imgur.com/3ARmOZr.jpg)

### Independencia
![](https://i.imgur.com/JBn2GhN.jpg)

#### bayes! 
### BAYES! 
## BAYES!

Contamos! --> asignamos probabilidades a que ocurran los n-grams que decidamos usar... y ya casi lo tenemos listo para su uso!


### Ejemplos de uso:

Ejemplo:

![](https://i.imgur.com/IlxTT9P.jpg)

Otro ejemplo.

![](https://i.imgur.com/T2ZUr6y.jpg)

## Imports


In [1]:
!pip install spacy

In [2]:
!python -m spacy download en_core_web_sm

    100% |████████████████████████████████| 37.4MB 48.1MB/s 

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_sm

    You can now load the model via spacy.load('en_core_web_sm')



In [0]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [0]:
doc = nlp('this is a test of a text processed')

In [5]:
[(t.text, t.pos_, t.dep_) for t in doc]

[('this', 'DET', 'nsubj'),
 ('is', 'VERB', 'ROOT'),
 ('a', 'DET', 'det'),
 ('test', 'NOUN', 'attr'),
 ('of', 'ADP', 'prep'),
 ('a', 'DET', 'det'),
 ('text', 'NOUN', 'pobj'),
 ('processed', 'VERB', 'acl')]

## Preprocesado de texto

In [0]:
ngram_type='char'
def get_vocabulary(corpus, ngram_type='char'):
  """
    :corpus: list of strings
  """
  if ngram_type == 'word':
    vocab = set([t.text for document in corpus for t in nlp(document)])
  else:
    vocab = []
    for doc in corpus:
      vocab += list(doc)
    vocab = set(vocab)
  return list(vocab)  

In [0]:
corpus =[
    'this is an apple', 'this is a document', 'we are in a Keepcoding course',
    'we are teaching a language model', 'the corpus will be super small',
    'this is also a test', 'we will see what we can do', 'this is an orange',
    'apples are not oranges', 'apples and oranges are fruits',
    'fruits are not vegatables', 'this is Sparta', 'is an orange a fruit'
        ]

In [0]:
vocab = get_vocabulary(corpus, ngram_type=ngram_type)

## Preparación del modelo

In [0]:
from collections import Counter, defaultdict

def get_unigrams(corpus):
  unigrams = defaultdict(int)
  for document in corpus:
    tokens = [t for t in document.split(" ")] #[t.text for t in nlp(document)]
    for token in tokens:
      unigrams[token]+=1
  return unigrams

def get_bigrams(corpus):
  bigrams = defaultdict(int)
  for document in corpus:
    tokens = [t for t in document.split(" ")]
    for ix in range(len(tokens)-1):
      bigram = " ".join(tokens[ix:ix+2])
      bigrams[bigram]+=1
  return bigrams


In [10]:
unigrams = get_unigrams(corpus=corpus)
list(unigrams.items())[:5]

[('this', 5), ('is', 6), ('an', 3), ('apple', 1), ('a', 5)]

In [11]:
bigrams = get_bigrams(corpus)
list(bigrams.items())[:5]

[('this is', 5), ('is an', 3), ('an apple', 1), ('is a', 1), ('a document', 1)]

In [0]:
def get_counts(corpus, ngram_range=(1,3), ngram_type='word'):
  grams = defaultdict(int)
  minrange, maxrange = ngram_range
  
  for document in corpus:
    if ngram_type=='word': 
      tokens = [t.text for t in nlp(document)]
      join_type = " ".join
    elif ngram_type == 'char':
      tokens = list(document)
      join_type = "".join
    else:
      raise 'wrong ngram_type (char/word)'
    for ix in range(len(tokens)-1):
      for nrange in range(minrange, maxrange+1):
        gram = join_type(tokens[ix:ix+nrange])
        grams[gram]+=1
  return grams
  
def compute_probs(counts, previous_counts, ngram_range, ngram_type):
  probs = {}
  for c in counts.keys():
    for pc in previous_counts.keys():
      k = " ".join(c.split(" ")[:ngram_range]) if ngram_type=='word' else "".join(list(c)[:ngram_range])
      if k == pc:
        probs[c]=counts[c]/previous_counts[pc]
  return probs

In [0]:

trigrams = get_counts(corpus, ngram_range=(3,3), ngram_type=ngram_type)
bigrams = get_counts(corpus, ngram_range=(2,2), ngram_type=ngram_type)
unigrams = get_counts(corpus, ngram_range=(1,1), ngram_type=ngram_type)

In [14]:
probs = compute_probs(trigrams, bigrams, 2, ngram_type)
list(probs.items())[:5]

[('thi', 0.8333333333333334),
 ('his', 0.8333333333333334),
 ('is ', 1.0),
 ('s i', 0.3125),
 (' is', 0.8333333333333334)]

In [15]:
probs = compute_probs(bigrams, unigrams, 1, ngram_type)
list(probs.items())[:5]

[('th', 0.4),
 ('hi', 0.75),
 ('is', 0.5789473684210527),
 ('s ', 0.7272727272727273),
 (' i', 0.12)]

Compute the probability of a sentence? Yes We Can!

Pero lo haremos en log space! Nada de multiplicar, porque cuando estemos trabajando con probabilidades muy pequenas y encima las estemos multiplicando, tendremos un problema de "underflow".

In [0]:
from math import log, exp, log2

def compute_sequence_log_prob(sequence, ngrams, ngram_range, e=0.00000, ngram_type='word'):
  probs = []
  if ngram_type=='word': 
    doc = [t.text for t in nlp(sequence)]
    join_type = " ".join
  elif ngram_type == 'char':
    doc = list(sequence)
    join_type = "".join
  else:
    raise 'wrong ngram_type (char/word)'
    
    
  for ix in range(len(doc)-ngram_range+1):
    ngram = join_type(doc[ix:ix+ngram_range])
    if ngram in ngrams:
      probs.append(ngrams[ngram])
    else:
      print(ngram)
      probs.append(e)
  logprob = sum([log(p) for p in probs])
  return logprob, exp(logprob)
    

In [17]:
compute_sequence_log_prob('this is an apple', ngrams=probs, ngram_range=2, ngram_type=ngram_type)

(-15.382574139320134, 2.0865698792834974e-07)

In [18]:
compute_sequence_log_prob('this is an orange', ngrams=probs, ngram_range=2, ngram_type=ngram_type)

(-15.821577512344115, 1.3451669592404814e-07)

In [19]:
compute_sequence_log_prob('this is a banana', ngrams=probs, ngram_range=2, ngram_type=ngram_type)

ba
na
na


ValueError: ignored

¿Qué acaba de pasar?

Ver P(x) = 0

## Smoothing - Laplace

En todos estos modelos, siempre hay que tener en cuenta, que pasa con todas esas palabras/tokens que no hemos visto en el entreno.

Que podemos hacer? Una solución muy simple se llama Laplace smoothing o add-one smoothing. Consiste en añadir +1 a todos los conteos, y así, hacemos ver que todas esas palabras que no hemos visto nunca, por lo menos han sido "vistas" una vez en nuestro train set.

No es ni mucho menos la mejor de las opciones, el más usado es [Kneser-Ney](http://www.foldl.me/2014/kneser-ney-smoothing/), os dejo el link aquí. No lo implementaremos, pero este tutorial contiene información muy valiosa, e intuiciones muy válidas.

In [23]:
def compute_probs_laplace(counts, previous_counts, ngram_range, ngram_type):
  probs = {}
  v = len(counts)
  for c in counts.keys():
    for pc in previous_counts.keys():
      k = " ".join(c.split(" ")[:ngram_range]) if ngram_type=='word' else "".join(list(c)[:ngram_range])
      if k == pc:
        probs[c]=(counts[c]+1)/(previous_counts[pc]+v)
  
  probs['<UNK>'] =(0+1)/(previous_counts[pc]+v)
  return probs

def compute_sequence_log_prob(sequence, ngrams, ngram_range, e=0.000000, ngram_type='word'):
  probs = []
  if ngram_type=='word': 
    doc = [t.text for t in nlp(sequence)]
    join_type = " ".join
  elif ngram_type == 'char':
    doc = list(sequence)
    join_type = "".join
  else:
    raise 'wrong ngram_type (char/word)'
    
    
  for ix in range(len(doc)-ngram_range+1):
    ngram = join_type(doc[ix:ix+ngram_range])
    if ngram in ngrams:
      probs.append(ngrams[ngram])
    else:
      probs.append(ngrams['<UNK>'])
  logprob = sum([log2(p) for p in probs])
  return logprob, pow(2,logprob), len(doc)

In [24]:
probs = compute_probs_laplace(trigrams, bigrams, 2, ngram_type)
list(probs.items())[:5]

[('thi', 0.03428571428571429),
 ('his', 0.03428571428571429),
 ('is ', 0.06666666666666667),
 ('s i', 0.032432432432432434),
 (' is', 0.03428571428571429)]

# ¿Cómo evaluamos un sistema de LM?

Normalmente usaríamos medidas cómo las que ya habéis visto: accuracy, precision, recall, F-1... En LM, no usamos ninguna de ellas. Usamos una medida que se llama perplexity. Perplejidad.

Una analogía muy buena que he leído últimamente es la siguiente. Cuándo nace un bebé, la perplejidad es muy alta, porque no tenemos ni idea de lo que va a decir (es un modelo sin entrenar). Cuando el bebé empieza a crecer, y a aprender sus primeras palabras y frases, la perplejidad va disminuyendo poco a poco, porque cada vez es más predecible saber que va a decir. Hasta que llega un momento, que os sorprende muy poco la estructura de sus frases porque ya ha aprendido un modelo de lenguaje muy particular. Este modelo no tiene que ser un castellano perfecto, o un inglés perfecto, es el lenguaje que habla en su día a día, en su casa, en el colegio, o donde sea que aprenda, justo igual que nuestros modelos. Si aprenden de un lenguaje muy coloquial, no podemos esperar que generen frases muy formales y viceversa. Pues esto es la perplejidad.

No entraremos en detalles de cómo se llego aquí, pero para que os hagáis una idea, viene de Claude Shannon y su teoría de la información. Lo que viene a decirnos es, como podemos guardar el máximo de información en los menores bits, es decir, quan bien somos capaces de predecir la siguiente palabra, y optimizar un modelo acorde a ello.

La fórmula de perplexity que implementaremos es la siguiente:

<div align="center">
![](https://i.imgur.com/0WwyBbc.png)
</div>




In [35]:
from math import pow

def perplexity(corpus, probs, ngrams_range, ngram_type):

  corpus_logprobs = 0
  N = 0
  for document in corpus:
    logprob, prob, n = compute_sequence_log_prob(document, probs, ngrams_range, ngram_type=ngram_type)
    corpus_logprobs += logprob
    N+=n
    print(corpus_logprobs)
  return pow(2,((-1/N)*corpus_logprobs))
                                                
  

In [34]:
len(probs)

170

In [36]:
perplexity(corpus, probs, ngrams_range=3, ngram_type=ngram_type)

-70.60789711773069
-157.86335613308836
-320.741211994787
-501.4340188944408
-677.9406238653603
-771.7777460956771
-917.6374611628173
-990.8179550226785
-1097.9591161545566
-1243.8860070148414
-1377.8864089884714
-1443.8655112897166
-1538.4686773281671


36.25301849638031

## Generar frases!

In [37]:
from random import choice
def compute_next(bigram, probs, ngram_range=2, ngram_type='word'):
  max_prob = 0
  candidates = []
  for key in probs.keys(): # argmax
    k = " ".join(key.split(' ')[:ngram_range]) if ngram_type=='word' else key[:ngram_range]
    if bigram == k:
      #print('key: "{}" -- {}'.format(key, probs[key]))
      if probs[key]>max_prob:
        max_prob = probs[key]
        candidates = [key]
      elif probs[key]==max_prob:
        candidates.append(key)
  if len(candidates)>0:
    candidate = choice(candidates)
    candidate = " "+" ".join(candidate.split(' ')[ngram_range:]) if ngram_type=='word' else candidate[ngram_range:]
    return (candidate, max_prob)
  else:
    return False
      
  

In [46]:
for _ in range(5):
    text = "this is" #
    prob = 1
    for i in range(50):
      bigram = text[-2:] if ngram_type=='char' else " ".join(text.split(' ')[-2:])
      # print('bigram:', bigram)
      next_p=compute_next(bigram, probs, ngram_type=ngram_type)
      # print('chosen: ', next_p)
      if next_p:
        text+=next_p[0]
      else:
        break
    print('Sample text: ',text)

Sample text:  this is are are are a fruits a tes a langes a Kee a doc
Sample text:  this is are are a tes are a fruits are a document
Sample text:  this is a langes are are a teachis a document
Sample text:  this is are are are are are a Keepcorange are are a fruit
Sample text:  this is a document
